<a href="https://colab.research.google.com/github/AngelTroncoso/proyecto_telecomx/blob/main/telecoX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [5]:
df = df[df['Churn'].str.strip() != '']

In [6]:
print(df['Churn'].unique())

['No' 'Yes']


In [7]:
df['Churn'].dtype

dtype('O')

In [8]:
df = pd.json_normalize(df.to_dict(orient='records'))
df

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7039,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7040,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7041,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [11]:
df.shape


(7043, 21)

In [12]:
rename_dict = {
    'customerID': 'ID_cliente',
    'Churn': 'Cancelado',
    'customer.gender': 'Género',
    'customer.SeniorCitizen': 'Adulto_mayor',
    'customer.Partner': 'Tiene_pareja',
    'customer.Dependents': 'Tiene_dependientes',
    'customer.tenure': 'Antigüedad_meses',

    'phone.PhoneService': 'Servicio_telefonía',
    'phone.MultipleLines': 'Líneas_múltiples',

    'internet.InternetService': 'Servicio_internet',
    'internet.OnlineSecurity': 'Seguridad_en_línea',
    'internet.OnlineBackup': 'Respaldo_en_línea',
    'internet.DeviceProtection': 'Protección_dispositivos',
    'internet.TechSupport': 'Soporte_técnico',
    'internet.StreamingTV': 'TV_en_streaming',
    'internet.StreamingMovies': 'Películas_en_streaming',

    'account.Contract': 'Tipo_contrato',
    'account.PaperlessBilling': 'Factura_sin_papel',
    'account.PaymentMethod': 'Método_pago',
    'account.Charges.Monthly': 'Cargo_mensual',
    'account.Charges.Total': 'Cargo_total'
}

In [13]:
df = df.rename(columns=rename_dict)

In [14]:
df.shape

(7043, 21)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_cliente               7043 non-null   object 
 1   Cancelado                7043 non-null   object 
 2   Género                   7043 non-null   object 
 3   Adulto_mayor             7043 non-null   int64  
 4   Tiene_pareja             7043 non-null   object 
 5   Tiene_dependientes       7043 non-null   object 
 6   Antigüedad_meses         7043 non-null   int64  
 7   Servicio_telefonía       7043 non-null   object 
 8   Líneas_múltiples         7043 non-null   object 
 9   Servicio_internet        7043 non-null   object 
 10  Seguridad_en_línea       7043 non-null   object 
 11  Respaldo_en_línea        7043 non-null   object 
 12  Protección_dispositivos  7043 non-null   object 
 13  Soporte_técnico          7043 non-null   object 
 14  TV_en_streaming         

In [16]:
df.isnull().sum()

,0
ID_cliente,0
Cancelado,0
Género,0
Adulto_mayor,0
Tiene_pareja,0
Tiene_dependientes,0
Antigüedad_meses,0
Servicio_telefonía,0
Líneas_múltiples,0
Servicio_internet,0


In [17]:
df['Cargo_total'] = df['Cargo_total'].str.replace(',', '.', regex=False)

In [18]:
df['Cargo_total'] = pd.to_numeric(df['Cargo_total'], errors='coerce')

In [19]:
df.head()

,ID_cliente,Cancelado,Género,Adulto_mayor,Tiene_pareja,Tiene_dependientes,Antigüedad_meses,Servicio_telefonía,Líneas_múltiples,Servicio_internet,...,Respaldo_en_línea,Protección_dispositivos,Soporte_técnico,TV_en_streaming,Películas_en_streaming,Tipo_contrato,Factura_sin_papel,Método_pago,Cargo_mensual,Cargo_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [20]:
df.dtypes

,0
ID_cliente,object
Cancelado,object
Género,object
Adulto_mayor,int64
Tiene_pareja,object
Tiene_dependientes,object
Antigüedad_meses,int64
Servicio_telefonía,object
Líneas_múltiples,object
Servicio_internet,object


In [21]:
for col in df.columns:
    print(f"\n📌 Columna: {col}")
    print(df[col].unique())


📌 Columna: ID_cliente
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']

📌 Columna: Cancelado
['No' 'Yes']

📌 Columna: Género
['Female' 'Male']

📌 Columna: Adulto_mayor
[0 1]

📌 Columna: Tiene_pareja
['Yes' 'No']

📌 Columna: Tiene_dependientes
['Yes' 'No']

📌 Columna: Antigüedad_meses
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]

📌 Columna: Servicio_telefonía
['Yes' 'No']

📌 Columna: Líneas_múltiples
['No' 'Yes' 'No phone service']

📌 Columna: Servicio_internet
['DSL' 'Fiber optic' 'No']

📌 Columna: Seguridad_en_línea
['No' 'Yes' 'No internet service']

📌 Columna: Respaldo_en_línea
['Yes' 'No' 'No internet service']

📌 Columna: Protección_dispositivos
['No' 'Yes' 'No internet service']

📌 Columna: Soporte_técnico
['Yes' 'No' 'No internet service']

📌 Columna: TV_en_streaming